# Amazon Product Co-Purchasing Network Analysis

## Assignment 3

In [ ]:
Team: Fireflies

Members: Chunhui Zhu, John Wong and Chunmei Zhu

Date: 2/25/2019

In [2]:
import pandas as pd
import os
print(os.listdir("."))

['.ipynb_checkpoints', 'amazon-meta.txt', 'Amazon0601.txt', 'Data620Group3_Proj3.ipynb']


### Extract Raw Data - Meta Data

In [4]:

#variables
id=[]
title=[]

id_str = ""
asin_str = ""
title_str = ""

reviewer=[]
count=0


myfile = open("amazon-meta.txt", encoding='utf-8' )

#extract amazon meta without customer review
output_file = open("amazon-small-meta.csv", encoding='utf-8',  mode = "w+")

#extract amazon review with product id
output_file3 = open("amazon-reviewer.csv", encoding='utf-8',  mode = "w+")
     

line = myfile.readline() 
    
while line:

    count+=1

    
    asin_str = ""
    title_str = "" #init value
    group_str = "" #init value
    categories_str = "" #init value
        

    if 'Id:' in line:

        reviewer.clear()
        
        numOfId = line[3:].strip()
        id_str = numOfId #id code as string
    
    
        #start after Id line: read in parallel
    
        line = myfile.readline()
    
     
        if 'ASIN:' in line:
        
            asin_str = line[5:].strip()
    
        line = myfile.readline()
        
        if 'title:' in line:
        
            name=line[8:].strip()
            name = name.replace(',',' ')
                        
            title_str = name

        line = myfile.readline()

        if 'group:' in line:
        
            name=line[8:].strip()
                        
            group_str = name

        
        line = myfile.readline() #read after salesrank
        
        line = myfile.readline() #read after similar
        
        line = myfile.readline() 
        

        if 'categories:' in line:
        
            name=line[14:].strip()
            name = name.replace(',',' ')                        
            categories_str = name


        output_file.writelines((id_str + "," + asin_str + "," + title_str + "," + group_str + "," + categories_str + "\n"))
        
           
              
    
    #skipping line by while loop
    line = myfile.readline()
    if 'cutomer:' in line: #only extract customer line        
        c_values = line.split(':')  
        #['2002-5-13  cutomer',' A2IGOA66Y6O8TQ  rating',' 5  votes','   3  helpful','   2']
                    
        name=c_values[1].replace('rating','')
        name = name.strip()
        reviewer.append(name)
        
        if id_str == '0':
            output_file3.writelines("1," + name + '\n') #unknown bug in first output, use "1,reviewer customer id"
        else:            
            output_file3.writelines(id_str + "," + name + '\n') #id,reviewer customer id
             

    
output_file.close()

output_file3.close()

myfile.close()


In [5]:
#verify File creation
print(os.listdir("."))

['.ipynb_checkpoints', 'amazon-meta.txt', 'amazon-reviewer.csv', 'amazon-small-meta.csv', 'Amazon0601.txt', 'Data620Group3_Proj3.ipynb']


### Extract Raw Data - Co-purchasing Data

There are commmets rows at the top of the co-purchasing file and it is '\t'.  we need to convert it to CSV file

In [10]:

myfile = open("Amazon0601.txt", encoding='utf-8' )

output_file = open("edges_product_id.csv", encoding='utf-8',  mode = "w+")


line = myfile.readline() 
    
while line:
    
    if line[0] != '#':
        output_file.writelines(line.replace('\t',','))
    
    line = myfile.readline()
    
    
myfile.close()
output_file.close()
    

# Narrow down co-purchasing data

Since there are over 1.7M rows of co-purchasing data over 400k products, we need to narrow down for more meaningful analysis.

We will focus on the top 100 most reviewed products.

In [7]:
#We are using Reducer code to aggregate the reviewer entries.

count = 0
perkeycount = 0
oldKey = None

myfile = open("amazon-reviewer.csv", encoding='utf-8' )

output_file = open("amazon-reviewerCountByProductID.csv", encoding='utf-8',  mode = "w+")


line = myfile.readline() 


line = myfile.readline() #skip header
        
while line:

    data_mapped = line.strip().split(",")
    if len(data_mapped) != 2:
        # Something has gone wrong. Skip this line.
        continue

    thisKey, thisReviewer = data_mapped
    
    if oldKey and oldKey != thisKey:
        output_file.writelines(str(oldKey) + "," + str(perkeycount) + "\n")
        perkeycount = 0

    oldKey = thisKey
    perkeycount += 1

    line = myfile.readline()
    
if oldKey != None:
        output_file.writelines(str(oldKey) + "," + str(perkeycount)  + "\n")    

myfile.close()
output_file.close()

In [8]:
#verify File creation
print(os.listdir("."))

['.ipynb_checkpoints', 'amazon-meta.txt', 'amazon-reviewer.csv', 'amazon-reviewerCountByProductID.csv', 'amazon-small-meta.csv', 'Amazon0601.txt', 'Data620Group3_Proj3.ipynb']


In [11]:
#next, we will create filter co-purchasing data and nodes id for the top 100 most reviewed products


reviewer_count_path = "amazon-reviewerCountByProductID.csv"

raw_edges_prodct_id_path = "edges_product_id.csv"

amazon_small_meta_path = "amazon-small-meta.csv"

top100_co_purchasing_savePath = "top100_edges.csv"

top100_node_savePath = "top100_nodes.csv"


###Create new co-purchasing list from top 100 reviewed product
reviewer_count = pd.read_csv(reviewer_count_path, index_col=False)

reviewer_count.columns = ["prod_id","review_count"]

top100_reviewer_count = reviewer_count.sort_values('review_count',ascending=False).head(n=100)


raw_edges_prodct_id = pd.read_csv(raw_edges_prodct_id_path, index_col=False)

raw_edges_prodct_id.columns = ["FromID","ToID"]

df_mergeFromNode = pd.merge(top100_reviewer_count, raw_edges_prodct_id, left_on = "prod_id", right_on = "FromID")


df_mergeToNode = pd.merge(top100_reviewer_count, raw_edges_prodct_id, left_on = "prod_id", right_on = "ToID")

df_new_coPurchasing = pd.concat([df_mergeFromNode, df_mergeToNode], ignore_index= True)

#output the new co-purchasing

df_new_edge = df_new_coPurchasing[['FromID','ToID']]

df_new_edge.columns = ['Source','Target']

df_new_edge.to_csv(top100_co_purchasing_savePath, index = None, header=True)

###Create node file for new co-purchasing list

amazon_small_meta =  pd.read_csv(amazon_small_meta_path, index_col=False)

amazon_small_meta.columns = ["prod_id","ASIN","title","group","category"]

in_fromid = set(df_new_coPurchasing['FromID'].tolist())

in_toid = set(df_new_coPurchasing['ToID'].tolist())

small_node_ids = in_fromid | in_toid

list_new_co_purchasing_prod_id = pd.DataFrame({'prod_id':list(small_node_ids)})

top100_nodes = pd.merge(list_new_co_purchasing_prod_id, amazon_small_meta, left_on = 'prod_id', right_on = 'prod_id')

#output the new node

top100_nodes_new = top100_nodes[['prod_id','title']]
 
top100_nodes_new.columns = ['id','label']

top100_nodes_new.to_csv(top100_node_savePath, index = None, header=True)

In [12]:
#verify File creation
print(os.listdir("."))

['.ipynb_checkpoints', 'amazon-meta.txt', 'amazon-reviewer.csv', 'amazon-reviewerCountByProductID.csv', 'amazon-small-meta.csv', 'Amazon0601.txt', 'Data620Group3_Proj3.ipynb', 'edges_product_id.csv', 'top100_edges.csv', 'top100_nodes.csv']
